In [1]:
qpath = r'../../quetzal/' # path to quetzal here
import sys
sys.path.insert(0, qpath)

data = r'inputs/'

# import class
from quetzal.io.gtfs_reader import importer

C:\Users\rgoix\AppData\Local\Continuum\anaconda3\envs\quetzal_env\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Read GTFS

In [3]:
feed = importer.GtfsImporter(path=data + r'bilbao.zip', dist_units='m')
feed = feed.clean()
feed.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20200310
5,num_routes,82
6,num_trips,3297
7,num_stops,498
8,num_shapes,0
9,num_frequencies,0


Frequency conversion currently work only for one specific service and date, and on one given time period.  
It computes the average headway over this time period.

### Restrict to one date and merge services

In [7]:
feed.stops['parent_station'] = None

In [10]:
feed.stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,2201,NaN,Anselma de Salces (2),NaN,43.266241,-2.922884,NaN,NaN,NaN,None
1,2417,NaN,Trauko (14),NaN,43.266699,-2.919561,NaN,NaN,NaN,None
2,3103,NaN,"Kepa Enbeitia ""Urretxindorra""",NaN,43.259403,-2.900301,NaN,NaN,NaN,None
3,6209,NaN,Gregorio la Revilla 1,NaN,43.263615,-2.939187,NaN,NaN,NaN,None
4,6210,NaN,Gregorio la Revilla 17,NaN,43.261591,-2.940032,NaN,NaN,NaN,None


In [8]:
feed = feed.restrict(dates=['20191210'])
feed.group_services()

In [11]:
feed.calendar_dates

,service_id,date,exception_type
0,1,20191210,1


In [12]:
feed.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,82
6,num_trips,3297
7,num_stops,498
8,num_shapes,0
9,num_frequencies,0


In [13]:
feed = feed.create_shapes()

In [14]:
feed.map_trips(
    feed.trips.groupby('route_id').first().trip_id.head(50)  # Map the first trip of each route
)

### Build patterns
In a non-frequency based GTFS, a route has many trips, each corresponding to a start at a specific time. It is necessary to group these trips in order to compute their headway. A group of trips is refered as a pattern.
The default method to build the patterns is by grouping trips based on their ordered list of stop_ids, without knowledge of time or duration.  
Other methods to build the patterns are available: by parent stations, or clusters. See example 4-advanced-patterns.ipynb

In [15]:
feed.build_patterns()

In [17]:
feed.trips

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,pattern_id
0,189,1,414668,Plaza Biribila,NaN,shape_009,189_0
1,189,1,414669,Plaza Biribila,NaN,shape_009,189_0
2,189,1,414670,Plaza Biribila,NaN,shape_009,189_0
3,189,1,414671,Plaza Biribila,NaN,shape_009,189_0
4,189,1,414672,Plaza Biribila,NaN,shape_009,189_0
...,...,...,...,...,...,...,...
3292,284,1,407798,Ametzola,NaN,shape_104,284_0
3293,284,1,407799,Ametzola,NaN,shape_104,284_0
3294,284,1,407800,Ametzola,NaN,shape_104,284_0
3295,284,1,407801,Ametzola,NaN,shape_104,284_0


In [19]:
feed.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,82
6,num_trips,3297
7,num_stops,498
8,num_shapes,124
9,num_frequencies,0


### Convert to frequencies

In [20]:
time_range = ['06:00:00', '09:00:00']  # time format must be HH:MM:SS
feed_f = feed.convert_to_frequencies(time_range=time_range)

100%|██████████| 76/76 [00:00<00:00, 110.50it/s]


In [22]:
feed_f.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,67
6,num_trips,76
7,num_stops,495
8,num_shapes,76
9,num_frequencies,76


The average headway computed is the interval length divided by the number of trip starts within the interval.
- a trip starting exactly at the start of the time range (6:00:00) is taken into account
- a trip starting exactly at the end of the time range (9:00:00) is not considered.

In [24]:
feed_f.trips

,pattern_id,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,189_0,189,1,189_0,Plaza Biribila,NaN,shape_009
1,190_0,190,1,190_0,Plaza Biribila,NaN,shape_095
2,191_0,191,1,191_0,Otxarkoaga,NaN,shape_087
3,192_0,192,1,192_0,Otxarkoaga,NaN,shape_020
4,193_0,193,1,193_0,Plaza Biribila,NaN,shape_003
...,...,...,...,...,...,...,...
71,256_0,256,1,256_0,Arenal,NaN,shape_032
72,277_0,277,1,277_0,Deustu,NaN,shape_010
73,278_0,278,1,278_0,Deustu,NaN,shape_006
74,283_0,283,1,283_0,Ametzola,NaN,shape_113


In [23]:
feed_f.frequencies

,trip_id,headway_secs,start_time,end_time
0,189_0,900,06:00:00,09:00:00
1,190_0,900,06:00:00,09:00:00
2,191_0,600,06:00:00,09:00:00
3,192_0,600,06:00:00,09:00:00
4,193_0,900,06:00:00,09:00:00
...,...,...,...,...
71,256_0,3600,06:00:00,09:00:00
72,277_0,900,06:00:00,09:00:00
73,278_0,1080,06:00:00,09:00:00
74,283_0,5400,06:00:00,09:00:00
